In [ ]:
### Install packages ###

# install.packages('tidyverse')

In [ ]:
# Load libraries

library(readr)
library(dplyr)

# library(tidyverse)
# library(tidyr)
# library(purrr) 
# library(glue)

In [ ]:
### Load data ###
data_dict_path <- './data/data_dictionary.csv'
encoded_data_path <- './data/data_encoded_04_24_2024.csv'

data_dict <- read_csv(data_dict_path, show_col_types = FALSE)

# Extract column names and descriptions from the first two rows before actually loading data
tmp_data <- read_csv(encoded_data_path, col_names = FALSE, show_col_types = FALSE)
col_names <- as.character(temp_data[1, ])
col_descriptions <- as.character(temp_data[2, ])

# Reload the data, skipping the first two rows and setting column names manually
encoded_data <- read_csv(encoded_data_path, skip = 2, col_names = col_names, show_col_types = FALSE)

# Store the column descriptions in a named list to create SPSS labels later
col_descriptions <- setNames(col_descriptions, col_names)

# spec(encoded_data)

# head(data_dict)
# head(encoded_data)
# print(col_descriptions)

In [ ]:
### FUNCTION FOR DECODING DATA

decode_data <- function(encoded_data, data_dict) {
    # Ensure data_dict has proper cols
    if (!("variable" %in% names(data_dict)) || !("label" %in% names(data_dict)) || !("value" %in% names(data_dict))) {
        stop("data_dict must contain 'variable', 'label', and 'value' columns.")
    }
    # Loop over the coded variables defined in data_dict and replace encoded values
    for (var in unique(data_dict$variable)) {
        if (var %in% names(encoded_data)) {
            mapping <- data_dict %>%
                       filter(variable == var)
            encoded_data[[var]] <- mapping$value[match(encoded_data[[var]], mapping$label)]
            } else {
            warning(paste("Variable", var, "not found in encoded_data."))
            }
    }
    return (encoded_data)
} 


x <- decode_data(encoded_data, data_dict)
x["Q6.4"]